In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix 
#from sklearn.cross_validation import train_test_split 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [ ]:
df=pd.read_csv(r"newtest.csv")
df=df.dropna()
data=df[['Ref.L.Cell.Thrp.Kbps.DL(kbit/s)','Ref.Avg.CQI(number)','Ref.DL.PRB.utilization(%)','L.UL.Interference.Avg(dBm)','Ref.L.RA.TA.UE.Index_All_Approx','Ish.L.User.Avg(number)']]

data=data.dropna()

# replace . and / with _ 
data.rename(columns={"Ref.L.Cell.Thrp.Kbps.DL(kbit/s)":"Cell.Thrp.Kbps.DL"},inplace=True)
data.columns=data.columns.astype(str).str.replace("[.]", "_")
data.columns=data.columns.astype(str).str.replace("[(]", "_")
data.columns=data.columns.astype(str).str.replace("[)]", "_")
data.columns=data.columns.astype(str).str.replace("[%]", "X")
print(data.columns)

data=data[data.loc[:,"Cell_Thrp_Kbps_DL"]!=0]

In [ ]:
# check max and min for create class labels
df['Ref.L.Cell.Thrp.Kbps.DL(kbit/s)'].max() # also check for .min()

47059.1311

In [ ]:
# check for max and mins for create X data labels. this process more generelize the program

print(data.Ref_Avg_CQI_number_.max())
print(data.Ref_Avg_CQI_number_.min())

print(data.Ref_DL_PRB_utilization_X_.max())
print(data.Ref_DL_PRB_utilization_X_.min())

print(data.L_UL_Interference_Avg_dBm_.max())
print(data.L_UL_Interference_Avg_dBm_.min())

print(data.Ref_L_RA_TA_UE_Index_All_Approx.max())
print(data.Ref_L_RA_TA_UE_Index_All_Approx.min())

print(data.Ish_L_User_Avg_number_.max())
print(data.Ish_L_User_Avg_number_.min())

In [ ]:
# categorize the X and Y data sets

data['CellT_category']=pd.cut(data['Cell_Thrp_Kbps_DL'], bins=np.arange(-10000, 60000, 12500),labels=np.arange(0,60000,12500).tolist())
data['CQI_category']=pd.cut(data['Ref_Avg_CQI_number_'], bins=np.arange(-2,20, 2),labels=np.arange(0,20,2).tolist())
data['PRB_category']=pd.cut(data['Ref_DL_PRB_utilization_X_'], bins=np.arange(-10,110, 10),labels=np.arange(0,110,10).tolist())
data['Interference_category']=pd.cut(data['L_UL_Interference_Avg_dBm_'], bins=np.arange(-126,6,6),labels=np.arange(-120,6,6).tolist())
data['RATA_category']=pd.cut(data['Ref_L_RA_TA_UE_Index_All_Approx'], bins=np.arange(-10,130,10),labels=np.arange(0,130,10).tolist())
data['User_category']=pd.cut(data['Ish_L_User_Avg_number_'], bins=np.arange(-25,350,25),labels=np.arange(0,350,25).tolist())

"""

data['CellT_category']=data.CellT_category.astype(int)
data['CQI_category']=data.CellT_category.astype(int)
data['PRB_category']=data.CellT_category.astype(int)
#data['L_UL_Interference_Avg_dBm_']=data.L_UL_Interference_Avg_dBm_.astype(int)
data['RATA_category']=data.CellT_category.astype(int)
data['User_category']=data.CellT_category.astype(int)
"""


"\n\ndata['CellT_category']=data.CellT_category.astype(int)\ndata['CQI_category']=data.CellT_category.astype(int)\ndata['PRB_category']=data.CellT_category.astype(int)\n#data['L_UL_Interference_Avg_dBm_']=data.L_UL_Interference_Avg_dBm_.astype(int)\ndata['RATA_category']=data.CellT_category.astype(int)\ndata['User_category']=data.CellT_category.astype(int)\n"

In [ ]:


Dset=data[['CQI_category','PRB_category','RATA_category','User_category','Interference_category','CellT_category']]



In [ ]:
Dset["CellT_category"].unique()

In [ ]:
#Dset.CellT_category.unique()

In [ ]:
# select X,Y data and Trsin test split
X=Dset.drop('CellT_category', axis=1)
Y=Dset['CellT_category']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.30, random_state=0)

In [ ]:

# create decission tree
dtree=DecisionTreeClassifier(max_depth=8,criterion='gini')
dtree.fit(X_train,Y_train)


In [ ]:
# predict and accuracy

predictions=dtree.predict(X_test)
accuracy=accuracy_score(Y_test,predictions)
accuracy

0.8058894230769231

In [ ]:
# confusion Metrix
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_test,predictions)
cm

array([[398, 178,   0,   0],
       [ 96, 935,   2,   0],
       [  0,  50,   4,   1],
       [  0,   0,   0,   0]])

In [ ]:
# heat map for confussion metrix

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(10,7))
sn.heatmap(cm, annot=True)
plt.xlabel('predict')
plt.ylabel('input')

In [ ]:
# draw Decission Tree

from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot

features=list(Dset.columns[0:5])
features

In [ ]:
# create classes
classes=np.arange(0,60001,12500).tolist()
classesX=[str(item) for item in classes]
classesX

In [ ]:

# create dot file and png file

dot_data=StringIO()
export_graphviz(dtree, out_file=dot_data,feature_names=features,filled=True,rounded=True,class_names=classesX)
graph=pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph[0].create_png())

